**<font color = blue size=4>爬取網易雲動態歌詞(更新版)</font>**<br>

In [1]:
import os
import re
import csv
import ast 
import json
import time
import urllib
import random
import requests
import datetime
import numpy as np
import pandas as pd
import urllib.parse
from bs4 import BeautifulSoup
from user_agent import generate_user_agent

In [2]:
# 匯入歌手的csv檔案
def import_csv(filepath):
    df_artists = pd.read_csv(filepath, delimiter = ',')
    return df_artists

In [3]:
# 取得歌手對應的歌曲清單，收集歌單內的歌曲ID
# 取得歌曲頁面url，存成list(song_url)
# https://segmentfault.com/a/1190000014948845
def get_songs(artist_id):
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'}
    r = requests.get('http://music.163.com/artist?id={}'.format(artist_id),headers=headers)
    song_ids = re.findall(r'song\?id=(\d+?)".+?</a>',r.text) #歌曲id列表
    song_titles = re.findall(r'song\?id=\d+?">(.+?)</a>',r.text)#歌名title列表
    song_url = []
    for i in range(len(song_ids)):
        song_id = song_ids[i]
        url = 'https://music.163.com/#/song?id={}'.format(song_id)
        song_url.append(url)
    
    # 根據特定歌手ID搜尋歌手名稱
    # https://music.163.com/#/artist?id=1045123
    df_artists_filter = df_artists.loc[df_artists["artist_id"] == artist_id]
    artist_name = df_artists_filter['artist_name'].values[0]

    artist_url = 'http://music.163.com/artist?id=' + str(artist_id) #歌手頁面連結
    
    return [song_ids, song_titles, song_url, artist_name, artist_url]

In [11]:
# 歌單的歌轉成dataframe並存成csv
# 亂碼使用encoding='utf_8_sig'解決
def get_songinfo(song_ids, song_titles, song_url, artist_name):
    df_songs = pd.DataFrame()
    artist_name_list = []
    for i in range(len(song_ids)):
        artist_name_list.append(artist_name)
    df_songs['Artist'] = pd.Series(artist_name_list)
    df_songs['SongID'] = pd.Series(song_ids)
    df_songs['SongTitle'] = pd.Series(song_titles)
    df_songs['SongURL'] = pd.Series(song_url)
    artist_name_new = artist_name.replace(' / ',' ')
    artist_name_new = validateTitle(artist_name_new)
    filepath = 'SingerData/SingerSongs/df_songs_' + artist_name_new + '.csv'
    df_songs.to_csv(filepath, encoding='utf_8_sig')
    return df_songs

In [5]:
# 檔案名稱所有非法字元都轉義：https://www.itread01.com/content/1549125182.html
def validateTitle(title):
    rstr = r"[\/\\\:\*\?\"\<\>\|]" # '/ \ : * ? " < > |'
    new_title = re.sub(rstr, "", title)  # 替換為空白
    return new_title

In [6]:
# 取得歌名和歌手
def get_name_artist(df_songs, count):
    artistname = df_songs['Artist'][count]
    artistname = validateTitle(artistname)
    songname = df_songs['SongTitle'][count]
    songname = validateTitle(songname)
    songid = df_songs['SongID'][count]
    songid = validateTitle(songid)
    #print(artistname + '_' + songname + '\n' + songid)
    return artistname, songname, songid

In [7]:
# 取得動態歌詞
def get_lyrics(songid):
    headers={'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
             'Accept-Encoding': 'gzip, deflate',
             'Accept-Language': 'zh-CN,zh;q=0.9',
             'Connection': 'keep-alive',
             'Cookie': '_iuqxldmzr_=32; _ntes_nnid=0e6e1606eb78758c48c3fc823c6c57dd,1527314455632; '
                       '_ntes_nuid=0e6e1606eb78758c48c3fc823c6c57dd; __utmc=94650624; __utmz=94650624.1527314456.1.1.'
                       'utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); WM_TID=blBrSVohtue8%2B6VgDkxOkJ2G0VyAgyOY;'
                       ' JSESSIONID-WYYY=Du06y%5Csx0ddxxx8n6G6Dwk97Dhy2vuMzYDhQY8D%2BmW3vlbshKsMRxS%2BJYEnvCCh%5CKY'
                       'x2hJ5xhmAy8W%5CT%2BKqwjWnTDaOzhlQj19AuJwMttOIh5T%5C05uByqO%2FWM%2F1ZS9sqjslE2AC8YD7h7Tt0Shufi'
                       '2d077U9tlBepCx048eEImRkXDkr%3A1527321477141; __utma=94650624.1687343966.1527314456.1527314456'
                       '.1527319890.2; __utmb=94650624.3.10.1527319890',
             'Host': 'music.163.com',
             'Referer': 'http://music.163.com/',
             'Upgrade-Insecure-Requests': '1',
             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                           'Chrome/66.0.3359.181 Safari/537.36'}
    url = 'http://music.163.com/api/song/lyric?id={}&lv=-1&kv=-1&tv=-1'.format(songid)
    user_agent = generate_user_agent()
    response = requests.get(url, headers={ 'user-agent': user_agent })
    # 隨機暫停 1~5 秒
    time.sleep(random.uniform(1,5))
    data = response.json()
    return data

In [10]:
# 下載動態歌詞，取代動態歌詞中的「’」符號，判斷回傳的lrc是否為空
# 解決UnicodeDecodeError: 'cp950'問題，加入encoding="utf-8"
def save_lrc_file(data, artistname, songname, count):
    content = data['lrc']['lyric']
    # lrc不為空
    if content != "":
        content = data['lrc']['lyric'].replace("’", "'")
        songname_new = songname.replace(' / ',' ')
        filepath = 'LrcData/'+ artist_name + '_' + songname_new +'.lrc'
        #檔名重複篩選
        if os.path.isfile(filepath):
            #檔案存在
            pass
        else:
            #檔案不存在
            with open(filepath, 'w', encoding="utf-8") as file:
                file.write(content)
            print(filepath)
            count += 1
    # lrc為空
    else:
        pass
    return count

In [13]:
# 主程式抓歌曲
count_list = []
base = 0 #base,base+100
for w in range(10,250):
    print('Round:' + str(w))
    df_artists = import_csv('SingerData/SingerListFinal_ID.csv')
    artist_id = df_artists['artist_id'][w]
    [song_ids, song_titles, song_url, artist_name, artist_url] = get_songs(artist_id)
    df_songs = get_songinfo(song_ids, song_titles, song_url, artist_name)
    count = 0
    for i in range(len(df_songs)):
        artistname, songname, songid = get_name_artist(df_songs, i)
        try:
            data = get_lyrics(songid)
            count = save_lrc_file(data, artistname, songname, count)    
        except:
            pass
    
    count_list.append(count)
    print('Artist: ' + artistname + ' / Count: ' + str(count))
    print('-'*80)

Round:10
LrcData/Alicia Keys_Empire State Of Mind.lrc
LrcData/Alicia Keys_Underdog.lrc
LrcData/Alicia Keys_If I Ain't Got You.lrc
LrcData/Alicia Keys_Fallin'.lrc
LrcData/Alicia Keys_Girl On Fire.lrc
LrcData/Alicia Keys_No One.lrc
LrcData/Alicia Keys_Show Me Love (Remix).lrc
LrcData/Alicia Keys_U Don't Know My Name (2F Remix).lrc
LrcData/Alicia Keys_Time Machine.lrc
LrcData/Alicia Keys_Like Home.lrc
LrcData/Alicia Keys_If I Ain’t Got You (Piano & Vocal Version).lrc
LrcData/Alicia Keys_Morning Light.lrc
LrcData/Alicia Keys_My Boo.lrc
LrcData/Alicia Keys_New Day.lrc
LrcData/Alicia Keys_Empire State of Mind (BPM 85).lrc
LrcData/Alicia Keys_Doesn't Mean Anything.lrc
LrcData/Alicia Keys_If i aint got you.lrc
LrcData/Alicia Keys_Us.lrc
LrcData/Alicia Keys_Fallin.lrc
LrcData/Alicia Keys_It's On Again.lrc
LrcData/Alicia Keys_Powerful.lrc
LrcData/Alicia Keys_A Woman's Worth.lrc
LrcData/Alicia Keys_Blended Family (What You Do For Love) .lrc
LrcData/Alicia Keys_One Thing.lrc
LrcData/Alicia Keys_Le

LrcData/Armin van Buuren_Turn It Up.lrc
LrcData/Armin van Buuren_This Is What It Feels Like (Extended Mix).lrc
LrcData/Armin van Buuren_Alone (Radio Edit).lrc
LrcData/Armin van Buuren_Great Spirit.lrc
LrcData/Armin van Buuren_Repeat After Me.lrc
LrcData/Armin van Buuren_Revolution.lrc
LrcData/Armin van Buuren_You Are.lrc
LrcData/Armin van Buuren_Another You.lrc
LrcData/Armin van Buuren_Face Of Summer (Omnia Remix).lrc
LrcData/Armin van Buuren_Freefall.lrc
LrcData/Armin van Buuren_Something Real.lrc
LrcData/Armin van Buuren_Make It Right.lrc
LrcData/Armin van Buuren_Gotta Be Love (Arston Extended Remix).lrc
LrcData/Armin van Buuren_Again (Armin van Buuren Remix).lrc
LrcData/Armin van Buuren_I Need You (Filatov & Karas Remix).lrc
LrcData/Armin van Buuren_Show Me Love.lrc
LrcData/Armin van Buuren_Wild Wild Son.lrc
LrcData/Armin van Buuren_If It Ain't Dutch (Extended Mix).lrc
LrcData/Armin van Buuren_Sex, Love & Water.lrc
LrcData/Armin van Buuren_Youtopia.lrc
LrcData/Armin van Buuren_Don't

LrcData/Axero_I Can't Do Without U.lrc
LrcData/Axero_Calling For You.lrc
LrcData/Axero_My Own Way.lrc
LrcData/Axero_Never Give Up.lrc
LrcData/Axero_Walk Away.lrc
LrcData/Axero_I'll Be Gone (feat. Katie McConnell).lrc
LrcData/Axero_Power.lrc
LrcData/Axero_What For.lrc
LrcData/Axero_I Can't Do Without You.lrc
LrcData/Axero_One More Step.lrc
LrcData/Axero_Lose You to Love Me.lrc
LrcData/Axero_Equality.lrc
LrcData/Axero_ We Own The Night .lrc
LrcData/Axero_Let's Get Lost.lrc
LrcData/Axero_Higher.lrc
LrcData/Axero_The Sweet Life (Tropical Version).lrc
LrcData/Axero_In Your Eyes.lrc
LrcData/Axero_Fahrenheit (Axero Remix).lrc
LrcData/Axero_Just Like You.lrc
LrcData/Axero_Runaway (U&I) (Axero Remix).lrc
Artist: Axero / Count: 20
--------------------------------------------------------------------------------
Round:22
LrcData/Barry Manilow_It Never Rains In Southern California.lrc
LrcData/Barry Manilow_Can't Smile Without You.lrc
LrcData/Barry Manilow_Mandy (Remastered).lrc
LrcData/Barry Manilo

LrcData/Billie Eilish_six feet under (cinnäfinn remix).lrc
LrcData/Billie Eilish_erbrokenhearts ( HuaoH Remix ).lrc
LrcData/Billie Eilish_Watch (Xie Remix).lrc
LrcData/Billie Eilish_Billie Eilish-Six Feet Under（M R U O  Billie Eilish  男巫婆 remix）.lrc
LrcData/Billie Eilish_Billie Eilish-Wish you were a gay（跳跳堂  Billie Eilish remix）.lrc
LrcData/Billie Eilish_Billie Eilish-bury a friend（跳跳堂  Billie Eilish remix）.lrc
Artist: Billie Eilish / Count: 49
--------------------------------------------------------------------------------
Round:28
LrcData/Blur_Ong Ong.lrc
LrcData/Blur_Fool's Day.lrc
LrcData/Blur_Tender.lrc
LrcData/Blur_Jets.lrc
LrcData/Blur_Rednecks.lrc
LrcData/Blur_Under the Westway (Acoustic).lrc
LrcData/Blur_Song 2.lrc
LrcData/Blur_Best Days.lrc
LrcData/Blur_Sweet Song.lrc
LrcData/Blur_My Terracotta Heart.lrc
LrcData/Blur_Girls and Boys.lrc
LrcData/Blur_Country House.lrc
LrcData/Blur_Parklife.lrc
LrcData/Blur_Coffee & TV.lrc
LrcData/Blur_Ghost Ship.lrc
LrcData/Blur_When The Cows 

LrcData/Britney Spears_Showdown.lrc
LrcData/Britney Spears_Do Something.lrc
LrcData/Britney Spears_Criminal.lrc
LrcData/Britney Spears_Selfish.lrc
LrcData/Britney Spears_Ooh Ooh Baby.lrc
LrcData/Britney Spears_Ooh La La.lrc
LrcData/Britney Spears_If U Seek Amy.lrc
LrcData/Britney Spears_Freakshow.lrc
LrcData/Britney Spears_What U See (Is What U Get).lrc
LrcData/Britney Spears_Me Against the Music (LP VersionVideo Mix).lrc
LrcData/Britney Spears_Coupure Électrique.lrc
LrcData/Britney Spears_Overprotected.lrc
LrcData/Britney Spears_Pretty Girls.lrc
LrcData/Britney Spears_Autumn Goodbye.lrc
LrcData/Britney Spears_Liar.lrc
LrcData/Britney Spears_Tom's Diner (Susanne Vega Cover).lrc
LrcData/Britney Spears_Perfume.lrc
LrcData/Britney Spears_Man On The Moon.lrc
Artist: Britney Spears / Count: 49
--------------------------------------------------------------------------------
Round:35
LrcData/Bruno Coulais_Les Choristes.lrc
LrcData/Bruno Coulais_Vois sur ton chemin.lrc
LrcData/Bruno Coulais_Ca

...

LrcData/ThimLife_By My Side.lrc
LrcData/ThimLife_Survive (Original Mix).lrc
LrcData/ThimLife_Dear Life (ThimLife Remix).lrc
LrcData/ThimLife_Stay The Night.lrc
LrcData/ThimLife_Never Walk Alone (Thimlife Remix).lrc
LrcData/ThimLife_False Alarm (Thimlife Remix).lrc
LrcData/ThimLife_Across the Water (Long Version).lrc
LrcData/ThimLife_Scars (Thimlife Remix).lrc
Artist: ThimLife / Count: 38
--------------------------------------------------------------------------------
Round:226
LrcData/Cheat Codes_No Promises.lrc
LrcData/Cheat Codes_Hold On (Radio Edit).lrc
LrcData/Cheat Codes_Pretty Girl (Cheat Codes X CADE Remix).lrc
LrcData/Cheat Codes_South of the Border (feat. Camila Cabello & Cardi B) [Cheat Codes Remix].lrc
LrcData/Cheat Codes_Shed a Light.lrc
LrcData/Cheat Codes_Balenciaga.lrc
LrcData/Cheat Codes_Stay With You.lrc
LrcData/Cheat Codes_Sober.lrc
LrcData/Cheat Codes_Put Me Back Together (feat. Kiiara).lrc
LrcData/Cheat Codes_Sex.lrc
LrcData/Cheat Codes_Shed A Light (Extended Versio

LrcData/Troye Sivan_There For You.lrc
LrcData/Troye Sivan_YOUTH.lrc
LrcData/Troye Sivan_for him..lrc
LrcData/Troye Sivan_BLUE.lrc
LrcData/Troye Sivan_Strawberries & Cigarettes.lrc
LrcData/Troye Sivan_i'm so tired....lrc
LrcData/Troye Sivan_FOOLS.lrc
LrcData/Troye Sivan_LOST BOY.lrc
LrcData/Troye Sivan_1999.lrc
LrcData/Troye Sivan_WILD.lrc
LrcData/Troye Sivan_My My My! (Cash Cash Remix).lrc
LrcData/Troye Sivan_EASE (Lontalius Remix).lrc
LrcData/Troye Sivan_Love Me Wrong.lrc
LrcData/Troye Sivan_SUBURBIA.lrc
LrcData/Troye Sivan_TALK ME DOWN.lrc
LrcData/Troye Sivan_HEAVEN.lrc
LrcData/Troye Sivan_Someone Like You.lrc
LrcData/Troye Sivan_SWIMMING POOLS.lrc
LrcData/Troye Sivan_Happy Little Pill.lrc
LrcData/Troye Sivan_Touch.lrc
LrcData/Troye Sivan_2099.lrc
LrcData/Troye Sivan_Fun.lrc
LrcData/Troye Sivan_for him (Live).lrc
LrcData/Troye Sivan_COOL.lrc
LrcData/Troye Sivan_Glittery (From The Kacey Musgraves Christmas Show).lrc
LrcData/Troye Sivan_BITE.lrc
LrcData/Troye Sivan_Papercut.lrc
LrcData

LrcData/Westlife_If I Let You Go.lrc
LrcData/Westlife_What About Now.lrc
LrcData/Westlife_Against All Odds.lrc
LrcData/Westlife_That's Where You Find Love.lrc
LrcData/Westlife_Evergreen.lrc
LrcData/Westlife_Written in the Stars.lrc
LrcData/Westlife_Miss You.lrc
LrcData/Westlife_Uptown Girl.lrc
LrcData/Westlife_Us Against the World.lrc
LrcData/Westlife_To Be Loved.lrc
LrcData/Westlife_When You Tell Me That You Love Me.lrc
LrcData/Westlife_Everybody Hurts.lrc
LrcData/Westlife_Fool Again.lrc
LrcData/Westlife_Leaving.lrc
LrcData/Westlife_Maybe Tomorrow.lrc
LrcData/Westlife_All Out Of Love.lrc
LrcData/Westlife_Without You.lrc
LrcData/Westlife_I Need You.lrc
LrcData/Westlife_Take Me There.lrc
LrcData/Westlife_I Have A Dream.lrc
LrcData/Westlife_Mandy.lrc
LrcData/Westlife_To Be with You.lrc
LrcData/Westlife_Why Do I Love You.lrc
LrcData/Westlife_Dance.lrc
LrcData/Westlife_One Last Time.lrc
LrcData/Westlife_Beautiful Tonight.lrc
LrcData/Westlife_L.O.V.E..lrc
LrcData/Westlife_Repair.lrc
LrcData

LrcData/Wynonie Harris_Good Rockin' Tonight.lrc
Artist: Wynonie Harris / Count: 1
--------------------------------------------------------------------------------
Round:244
LrcData/XXXTENTACION_NUMB.lrc
LrcData/XXXTENTACION_Jocelyn Flores.lrc
LrcData/XXXTENTACION_whoa (mind in awe).lrc
LrcData/XXXTENTACION_Sad!.lrc
LrcData/XXXTENTACION_Everybody Dies In Their Nightmares.lrc
LrcData/XXXTENTACION_changes.lrc
LrcData/XXXTENTACION_Ex Bitch.lrc
LrcData/XXXTENTACION_Changes (Seizure Remix).lrc
LrcData/XXXTENTACION_the remedy for a broken heart (why am I so in love).lrc
LrcData/XXXTENTACION_Moonlight.lrc
LrcData/XXXTENTACION_Scared of the Dark.lrc
LrcData/XXXTENTACION_Hope.lrc
LrcData/XXXTENTACION_ Love.lrc
LrcData/XXXTENTACION_difference (interlude).lrc
LrcData/XXXTENTACION_insecure (ocean mix).lrc
LrcData/XXXTENTACION_what are you so afraid of.lrc
LrcData/XXXTENTACION_Carry On.lrc
LrcData/XXXTENTACION_bad vibes forever.lrc
LrcData/XXXTENTACION_changes (Remix).lrc
LrcData/XXXTENTACION_Fallin

LrcData/Backstreet Boys_We've Got It Goin' On.lrc
LrcData/Backstreet Boys_Yes I Will.lrc
LrcData/Backstreet Boys_Christmas Time.lrc
LrcData/Backstreet Boys_Siberia.lrc
LrcData/Backstreet Boys_Bigger.lrc
LrcData/Backstreet Boys_Passionate.lrc
LrcData/Backstreet Boys_Breathe.lrc
LrcData/Backstreet Boys_Just Like You Like It.lrc
Artist: Backstreet Boys / Count: 49
--------------------------------------------------------------------------------
Round:248
LrcData/Zedd_I Want You To Know (Hella x Pegato Remix) .lrc
LrcData/Zedd_Beautiful Now.lrc
LrcData/Zedd_Stay.lrc
LrcData/Zedd_I Want You to Know.lrc
LrcData/Zedd_The Middle.lrc
LrcData/Zedd_Ignite.lrc
LrcData/Zedd_Break Free.lrc
LrcData/Zedd_Good Thing.lrc
LrcData/Zedd_365.lrc
LrcData/Zedd_Papercut.lrc
LrcData/Zedd_Clarity.lrc
LrcData/Zedd_Starving.lrc
LrcData/Zedd_Happy Now.lrc
LrcData/Zedd_I WaNt U To KnOw (marshmello Remix).lrc
LrcData/Zedd_Spectrum.lrc
LrcData/Zedd_Lost In Japan (Remix).lrc
LrcData/Zedd_Stay the Night.lrc
LrcData/Zedd_

In [12]:
# 有下載到的lrc數量
print(count_list)
print('LRC count:',len(count_list))

# 把每一位歌手抓到的lrc儲存起來並寫入csv
with open("CloudMusic_csv/df_songcount.csv", "a", encoding="utf-8", newline='') as csvfile:
    writer = csv.writer(csvfile)
    Round = str(base) + '-' + str(base+100)
    Count_list = count_list
    #第一次寫入
    #writer.writerow(['Round', 'Count_list'])
    #之後寫入
    writer.writerow([Round, Count_list])

df_songcount = pd.read_csv('CloudMusic_csv/df_songcount.csv', delimiter = ',')
df_songcount

[48, 23, 48, 30, 38, 15, 4, 3, 29, 0, 5, 2, 33, 0, 31, 0, 1, 4, 7, 5, 1, 35, 0, 0, 3, 1, 0, 6, 3, 11, 2, 20, 4, 0, 1, 0, 1, 19, 0, 0, 3, 1, 2, 8, 10, 5, 2, 0, 0, 4, 17, 1, 0, 27, 0, 37, 9, 0, 2, 1, 0, 0, 1, 4, 11, 1, 0, 0, 6, 0, 7, 20, 0, 1, 0, 15, 3, 0, 1, 29, 1, 0, 2, 2, 0, 1, 0, 0, 1, 1, 2, 6, 2, 1, 2, 1, 3, 0, 1, 2]
LRC count: 100


,Round,Count_list
0,0-100,"[36, 44, 24, 5, 50, 38, 6, 43, 32, 40, 50, 26,..."
1,100-200,"[46, 45, 31, 12, 38, 39, 21, 46, 5, 46, 30, 18..."
2,200-300,"[49, 46, 49, 49, 28, 34, 5, 49, 9, 24, 49, 16,..."
3,300-400,"[25, 47, 48, 37, 47, 30, 38, 48, 50, 29, 1, 32..."
4,400-500,"[50, 50, 20, 6, 5, 6, 27, 1, 49, 3, 48, 34, 17..."
5,500-600,"[46, 27, 14, 46, 24, 10, 25, 20, 1, 6, 29, 13,..."
6,600-700,"[50, 48, 45, 43, 49, 3, 43, 49, 5, 22, 1, 30, ..."
7,700-800,"[37, 6, 24, 19, 21, 7, 41, 2, 1, 44, 9, 3, 8, ..."
8,800-900,"[49, 9, 2, 5, 6, 6, 27, 36, 19, 50, 1, 20, 35,..."
9,900-1000,"[49, 46, 47, 49, 39, 12, 48, 48, 14, 48, 22, 5..."
